## Imports

In [2]:
import pandas as pd
import re
import timeit
import seaborn as sns
import lightgbm as lgb
import numpy as np
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import sklearn.metrics as sm
from sklearn.ensemble import IsolationForest
import plotly.express as px
import numpy as np
import warnings
import datetime
import xgboost as xgb
import math
import joblib
import matplotlib.pyplot as plt
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV
import tsaug
from tsaug.visualization import plot
import smogn
from catboost import CatBoostRegressor
import optuna.integration.lightgbm as lgb
import optuna


warnings.filterwarnings("ignore")

#Imports TFG Original
from prettytable import PrettyTable
from os import listdir
from os.path import isfile, join
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score , mean_squared_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from yellowbrick.model_selection import FeatureImportances
from sklearn.linear_model import LinearRegression



In [3]:
pathAlt = "Resultats/GridSearchManual/AltTer/"
pathBaix = "Resultats/GridSearchManual/BaixTer/"

# Visualització Resultats (Alt Ter)

In [4]:
normal = pd.read_csv(pathAlt + 'DfAltTerNormalResults.csv',index_col=0)
anomaliesPerPrecipitacions = pd.read_csv(pathAlt + 'DfAltTerAnomaliesPrecipitacionsResults.csv',index_col=0)
anomaliesPerCabal = pd.read_csv(pathAlt + 'DfAltTerAnomaliesCabalResults.csv',index_col=0)
anomaliesPerPrecipitacionsMedian = pd.read_csv(pathAlt + 'DfAltTerAnomaliesPrecipitacionsMedianResults.csv',index_col=0)
smogn = pd.read_csv(pathAlt + 'DfAltTerAnomaliesSMOGNResults.csv',index_col=0)
standard = pd.read_csv(pathAlt + 'DfAltTerStandardResults.csv',index_col=0)


In [5]:
normal

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,13.325342,180.976220,9.219331,28.987690,300.233887
1,Random Forest,6.956233,123.271903,9.629026,28.350500,180.740651
2,Random Forest,16.533634,125.886100,9.370972,29.625492,190.717715
3,XGBoost,15.122113,186.361240,10.717727,32.196245,286.496034
4,XGBoost,8.737360,113.946922,10.066348,32.198160,137.757882
5,XGBoost,15.649445,122.827862,10.075246,34.644820,167.356655
6,LightGBM,16.448586,243.509511,11.112875,30.986329,351.041222
7,LightGBM,9.388974,223.967227,10.863894,29.036639,252.160040
8,LightGBM,23.332985,210.642158,10.320408,32.474603,270.915519
9,CatBoost,16.118436,230.771458,9.367525,32.134082,371.924378


In [6]:
anomaliesPerPrecipitacions

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,62.204700,175.207701,8.747487,34.469956,295.808743
1,Random Forest,40.878922,121.035164,8.323429,30.971669,180.762630
2,Random Forest,86.516198,133.205551,8.745108,30.420243,198.520262
3,XGBoost,62.135716,175.430880,10.614323,37.341214,267.484429
4,XGBoost,64.742657,118.995154,10.139152,35.041535,150.392294
5,XGBoost,86.695800,140.966296,10.262134,40.831022,194.987991
6,LightGBM,62.798115,178.885199,10.979583,36.928708,265.813977
7,LightGBM,51.219964,116.371338,9.909083,36.378399,129.857206
8,LightGBM,84.879700,149.711599,11.049109,37.448208,155.801751
9,CatBoost,62.245170,197.603179,11.314402,34.729828,273.722507


In [7]:
anomaliesPerCabal

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,12.522484,178.773728,8.608339,30.878961,303.161771
1,Random Forest,18.490588,186.511380,8.485605,25.274670,307.146817
2,Random Forest,22.507418,126.769302,8.735204,26.927410,181.761266
3,XGBoost,13.199104,163.627042,9.653184,33.096622,254.485674
4,XGBoost,17.837124,169.090247,8.938451,31.489545,271.910103
5,XGBoost,21.726961,117.758082,9.696913,31.863347,176.741200
6,LightGBM,13.332773,239.528498,9.102943,30.718250,299.551898
7,LightGBM,24.031015,240.980838,9.657518,30.792779,396.058881
8,LightGBM,29.321849,183.115886,9.582695,29.995781,217.643494
9,CatBoost,12.896412,207.915650,8.253953,33.287425,379.943630


In [8]:
smogn

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,274.724685,224.711091,66.398404,42.674309,418.556945
1,Random Forest,37.874855,51.922968,6.591570,14.513227,49.528781
2,Random Forest,30.762652,52.744101,7.345919,12.330821,55.427595
3,XGBoost,270.518353,222.482417,59.630808,45.775667,413.408513
4,XGBoost,36.328844,60.935652,9.586234,13.143738,55.030221
5,XGBoost,34.676514,64.602217,11.575111,14.509703,69.567197
6,LightGBM,270.503878,219.174615,56.824117,38.861933,419.241295
7,LightGBM,37.479670,67.697768,9.119133,17.253036,42.303215
8,LightGBM,31.466819,57.790795,9.113156,14.094735,48.037058
9,CatBoost,270.159830,218.967343,36.200492,32.637664,416.241236


In [9]:
anomaliesPerPrecipitacionsMedian

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,16.644685,213.614895,8.916264,28.640805,373.252222
1,Random Forest,7.341676,119.881981,9.471746,26.689728,180.874718
2,Random Forest,17.081599,128.278084,9.633236,29.252711,180.234213
3,XGBoost,14.648717,172.068679,8.850937,30.282918,257.595334
4,XGBoost,9.264407,118.842701,10.120644,33.564524,139.235722
5,XGBoost,16.813980,122.607032,10.205873,30.998874,181.552110
6,LightGBM,15.124106,238.328615,9.282335,31.336874,331.388693
7,LightGBM,9.920785,194.829779,10.327458,28.617599,233.873423
8,LightGBM,24.659499,223.549748,9.939520,30.524272,244.562859
9,CatBoost,17.678960,242.946700,10.004070,31.140453,366.793289


In [5]:
standard

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,8.069291,249.358096,8.276929,28.615760,478.227554
1,Random Forest,4.538592,221.681936,9.178022,23.411833,410.530074
2,Random Forest,6.452600,224.125082,8.698898,21.856674,417.480949
3,XGBoost,7.861023,202.720254,8.485928,26.633486,357.103092
4,XGBoost,4.190788,194.614060,11.738527,22.264948,329.077606
5,XGBoost,7.126559,208.573996,8.849939,28.790826,373.868281
6,LightGBM,7.829297,233.271286,8.629657,27.260631,431.661061
7,LightGBM,4.717233,242.778436,10.712867,25.118754,459.727238
8,LightGBM,7.738690,240.351762,10.302029,30.222978,455.236277
9,CatBoost,8.442543,251.424787,10.696561,30.530444,484.287984


# Visualització Resultats (Baix Ter)

In [10]:
normal = pd.read_csv(pathBaix + 'DfBaixTerNormalResults.csv',index_col=0)
anomaliesPerPrecipitacions = pd.read_csv(pathBaix + 'DfBaixTerAnomaliesPrecipitacionsResults.csv',index_col=0)
anomaliesPerCabal = pd.read_csv(pathBaix + 'DfBaixTerAnomaliesCabalResults.csv',index_col=0)
anomaliesPerPrecipitacionsMedian = pd.read_csv(pathBaix + 'DfBaixTerAnomaliesPrecipitacionsMedianResults.csv',index_col=0)
smogn = pd.read_csv(pathBaix + 'DfBaixTerSMOGNResults.csv',index_col=0)
standard = pd.read_csv(pathBaix + 'DfBaixTerStandardResults.csv',index_col=0)

In [11]:
normal

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,17.086738,253.261427,11.984013,46.915963,472.220618
1,Random Forest,NaN,4.113534,243.212487,9.082267,30.841298,442.543601
2,Random Forest,NaN,31.985423,240.234369,12.371092,43.326048,438.884246
3,XGBoost,NaN,17.786427,222.268612,19.559925,63.826117,372.435278
4,XGBoost,NaN,3.799999,180.653559,18.227264,59.725614,247.985737
5,XGBoost,NaN,38.248166,215.485366,16.985982,58.140885,371.119314
6,LightGBM,NaN,18.487053,252.015240,15.681580,49.282741,484.547666
7,LightGBM,NaN,3.869862,241.409948,16.757743,56.044583,458.204015
8,LightGBM,NaN,38.921610,217.517939,24.610083,60.094830,386.115816
9,ExtraTrees,NaN,18.163395,240.063931,10.806208,38.099536,448.913590


In [12]:
anomaliesPerPrecipitacions

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,58.594923,261.185741,12.332992,36.700665,503.573232
1,Random Forest,NaN,60.299800,243.527933,22.120836,30.243184,461.207480
2,Random Forest,NaN,87.567358,237.517841,10.439806,35.318875,445.394210
3,XGBoost,NaN,59.523277,272.395968,10.662858,50.143187,520.892961
4,XGBoost,NaN,60.112065,250.014458,28.004937,57.530949,466.905376
5,XGBoost,NaN,104.864455,234.409406,10.807298,37.215912,437.456522
6,LightGBM,NaN,63.781728,267.915195,6.050012,37.753617,512.539184
7,LightGBM,NaN,57.640691,253.315990,14.643865,43.772544,473.882926
8,LightGBM,NaN,132.172777,218.206987,19.037581,53.824481,387.060032
9,ExtraTrees,NaN,68.699403,252.308981,11.362359,27.020341,479.631578


In [13]:
anomaliesPerCabal

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,12.026263,240.749217,12.087946,32.299360,457.578480
1,Random Forest,NaN,21.571797,239.698060,10.232101,30.806105,444.571155
2,Random Forest,NaN,44.823241,227.924666,9.445256,35.828918,413.239241
3,XGBoost,NaN,12.980120,183.959073,10.524870,41.981088,291.933816
4,XGBoost,NaN,22.432992,231.081734,10.018385,32.578681,441.406761
5,XGBoost,NaN,49.383515,201.820376,11.510899,51.439297,348.140822
6,LightGBM,NaN,11.641638,245.503513,9.850309,39.141029,455.839593
7,LightGBM,NaN,23.575951,234.113174,10.616682,42.635569,433.646928
8,LightGBM,NaN,48.468752,215.479377,9.572034,40.552840,361.305200
9,ExtraTrees,NaN,14.133724,239.026248,8.459330,32.270395,450.733885


In [14]:
anomaliesPerPrecipitacionsMedian

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,51.207235,281.739846,15.289781,51.397367,542.918736
1,Random Forest,NaN,14.511194,242.842705,11.837545,42.768238,440.387044
2,Random Forest,NaN,36.853791,231.545996,11.584709,39.881222,424.409755
3,XGBoost,NaN,51.382451,291.060736,16.759837,57.681791,563.309322
4,XGBoost,NaN,18.989539,228.660629,18.022925,63.720473,402.434901
5,XGBoost,NaN,41.686010,238.473139,14.372658,56.920976,470.803904
6,LightGBM,NaN,51.013117,284.460437,14.959010,52.817875,549.508491
7,LightGBM,NaN,21.219839,240.574383,15.669918,50.088520,434.315928
8,LightGBM,NaN,40.931791,252.721044,17.273454,51.916114,487.210579
9,ExtraTrees,NaN,51.992508,282.801791,12.110789,50.473044,543.027513


In [15]:
smogn

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,267.939969,249.238979,69.207816,45.086317,471.876633
1,Random Forest,NaN,28.283388,228.417532,8.861116,24.896111,424.546370
2,Random Forest,NaN,23.320931,236.331311,10.767297,30.529064,440.605917
3,XGBoost,NaN,250.804818,246.887240,74.922319,67.436978,445.579473
4,XGBoost,NaN,30.217098,258.130810,20.749087,19.584785,485.679021
5,XGBoost,NaN,24.289147,271.404130,16.210190,47.738132,490.799836
6,LightGBM,NaN,251.958914,250.211057,56.482608,47.804110,454.641401
7,LightGBM,NaN,28.695541,256.948088,14.749860,33.310326,477.043149
8,LightGBM,NaN,26.074668,245.892925,10.534807,34.127640,450.033680
9,ExtraTrees,NaN,265.397006,241.774266,57.777921,39.056646,462.512814


In [16]:
standard

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,11.717206,267.195682,12.389123,44.918508,500.315343
1,Random Forest,NaN,2.871385,261.330399,12.164835,47.867470,490.487807
2,Random Forest,NaN,26.693478,264.463912,13.753476,51.502289,491.794132
3,XGBoost,NaN,11.606437,272.624551,16.784416,57.224310,525.185711
4,XGBoost,NaN,2.905904,249.332983,14.133146,59.170095,470.523361
5,XGBoost,NaN,26.379639,247.316045,17.437737,61.186497,443.158947
6,LightGBM,NaN,13.467293,279.545668,14.742609,56.662969,532.950968
7,LightGBM,NaN,2.810701,248.079780,16.488837,54.824501,460.289259
8,LightGBM,NaN,28.839558,249.466837,14.096890,56.109499,456.568092
9,ExtraTrees,NaN,10.822045,271.043501,11.073848,43.030407,511.476101
